<h1><center>Project</center></h1>
<h2><center>Micro-452 : Basics of Mobile Robotics</center></h2>
<h3><center>Group 11</center></h3>
<h3><center>Benoist Marguerite, Chang Chun-Tzu, Maradan Théodore, Tambourin Noé</center></h3>

## 1. Introduction

This report presents the project done in the Basics of Mobile Robotics class. The project consists in programming a robot (the "Thymio") to navigate in an environment containing a set of (expected and unexpected) obstacles.

To do so, the robot should first be able to build a global map of its environment using a camera (<b>vision</b>). From the map, the robot should determine the shortest path from its starting point to its goal (<b>global naviagation</b>). Then, <b>filtering</b> the information coming from the camera and from the built-in sensors the robot should travel to a goal point set anywhere on the map using <b>motor control</b>. Finally, the robot should avoid any unexpected obstacle on its path using <b>local navigation</b>.

<font color='red'>INCLUDE DIAGRAM PROJECT</font>

The sections below describe in detail our solution to the problem and, in particular, the choices and assumptions made through the realization of this project.

## 2. Environment Choice

The environment constituting the global map was chosen to be blank with 2D black obstacles (obstacles without thickness). The contrast between the map and the obstacles allows for a better detection of the obstacles by the camera. In contrast, the unexpected obstacles are 3D (non-null thickness).

This distinction ensures that the obstacles detected by the proximity sensors are unexpected obstacles and avoids confusion with obstacles being part of the global map.

<font color='red'>INCLUDE FIGURE ENVIRONMENT</font>

In order to identify the robot and the goal point, ArUco markers are used. These synthetic markers are easy to identify using an appropriate library (OpenCV). Moreover, they ensure a reliable robot pose acquisition by the camera.

<font color='red'>INCLUDE FIGURE ARUCO MARKER</font>

Finally, the camera is fixed above the global map so that it sees the entire environment while avoiding as much as possible distortions if the map due to the camera not being parallel with the plane of the map.

<font color='red'>INCLUDE FIGURE ENVIRONMENT + CAMERA</font>

## 3. Vision

## 4. Global Navigation

As stated in the introduction, the goal of the global navigation is to compute the shortest path from the start to the goal using the image acquired by the camera. To do so, a library (PyVisGraph) is used. PyVisGraph first builds a visibility graph from the vertices of the obstacles and, then, uses Dijsktra's algorithm to find the shortest path.

The coordinates of the start and goal as well as the coordinates of the vertices are given as output of the vision part. The vertices are formatted into an array which composition is as follows : Each row corresponds to a different obstacle and each element in the row corresponds to the coordinates (x & y) of the vertices of the obstacle.

Knowing the vertices and the obstacles they form, it is possible to build a visibility graph. The latter determines which vertices are visible from each vertex of each obstacle. Having the coordinates of each vertex, the Euclidean norm of the segment linking two vertices can be used to determine their distance to each other. The problem of collision with obstacles is avoided by adding margins on the obstacles dimensions in the vision part directly.

<font color='red'>INCLUDE FIGURE VISIBILITY GRAPH</font>

As mentioned earlier, Dijkstra's algorithm is used to compute the shortest path. Besides its easy implementation, this method has several advantages: It guarantees to find the shortest path and is relatively efficient (and can, therefore, be applied to fairly large problems). However, the path computed is not optimal (the robot could, sometimes, have a more direct path than traveling from vertex to vertex).

The algorithm visits each vertex once beginning from the goal point to which it assigns a distance of zero. It computes the distance from the vertex it is currently visiting to each of the vertices preceding it (the vertices being linked to the current one in the visibility graph on the side of the starting point) and attributes them the distance computed. When the distance from the goal to a vertex is computed several times, only the shortest distance to it is attributed to this specific vertex. Once the distances are computed, the algorithm moves to the unvisited node with the lowest distance attached to it and repeats the process until reaching the starting point. 

Finally, the total distance from the starting point to the goal using the shortest path is the distance assigned to the starting point. As the algorithm keeps track of the shortest distance to each of the vertices at each stage, the shortest path can be found easily.

<font color='red'>INCLUDE PSEUDO CODE DIJKSTRA</font>

## 5. Filtering

The Kalman function is dedicated to fusing and filtering the output of two sensors (the camera and the sensors on the motors) in order to improve the precision of the localization of the thymio. The Kalman filter is implemented using the correponsding functions from the filterpy library. To do so, we determine 5 matrixes that describe our system:

-	The matrix F describes the relation between the current state and the next state. 
-	The matrix H describes the relation between the state and the measurements
-	The matrix P is a state covariance matrix. It is a diagonal matrix which elements are the variance of each state.
-	The matrix P gives information about the noise of each measurement. It is also a diagonal matrix, which elements are the variance of each measured value.
-	The matrix Q is related to the process noise (which we decided to set as discrete white noise).

The next step consists in using the different sensors to obtain measurements.

Two functions are used to do a prediction about a state, to update the measurements and to filter the state. The function output is the filtered state.

Our state robot_state has four components: x and y position and x and y speed ($v_x$ and $v_y$). Our measurements also have four components: x and y position measurements from the camera that is updated in each iteration of the main loop (as a new image is being taken) as well as $v_x$ and $v_y$ measured directly on the motors. 

The motor speeds are given by left and right speed. We use the direction of the robot which is a global variable, to transform them into x and y speeds. 

### 5.1 Hidden camera scenario

We wanted our Kalman to still update the state of the robot correctly even when the camera is temporarily hidden. To do so, we add a condition to see if a new image has been taken. If no new image is available, the Kalman doesn’t use the position measurement from the camera which is outdated and would false the filtering. Instead, we use the current x and y position of the robot from robot_state instead of the position measurement as a temporary measure. Once the camera is back on, the position measurements from the camera will be used again. 

### 5.2 Kalman parameters 

The variances of our state position and speed were tuned by trial and error during the testing of the Kalman filter with the rest of the code.

The variance of the measurements, both from the camera and the motors were determined experimentally. We implemented a python code that records 1000 measurements from either the camera or the motor sensors and calculates the variance. Those measurements were made while the Thymio kept either the same position or the same speed, depending on the value measured by the sensor. The variance found varied each time we run our code. So we dediced to take the mean of the several resulting variances.

<font color='red'>REMARK : BETTER EXPLAIN 1000 MEASUREMENTS</font>


## 6. Motor Control

## 7. Local Navigation

Navigation is the function that controls the state of the robot and applies the corresponding speed on the two motors. It is called once in each iteration of the main loop. There are two states: state 0 and state 1.

In state 0, the robot is in global navigation mode and follows the global_path trajectory given as input to the navigation function. When in state 0, the global_nav() function is called with the path as input. The speeds given by global_nav() are then applied to the motors.

<font color='red'>CAREFUL ABOUT SEPARATION BETWEEN MOTOR CONTROL AND LOCAL NAVIGATION</font>

In state 1, the robot is avoiding a 3D obstacle that was added and, therefore, not recorded on the global map. Each time navigation() is called, the prox.horizontal values of the thymio are read. We compute the normalized weighted sum of the two sensor values on the left (prox[0] and prox[1]) on the one hand and of the two right sensor values (prox[3] and prox[4]). The sum is weighted (weight of 1/2 for the proximity sensor on the far right/center right) to give more importance on the measurements given by sensors closer to the central axis of the robot. Indeed, an obstacle being closer to this axis will require a higher deviation from the initial path to avoid it.

The velocity of the left (right) motor is computed by subtracting the result of the weighted sum of measurements given by the left (right respectively) proximity sensors multiplied by a coefficient and subtracted to a constant value. This way, the left prox sum is contributing to the left motor speed and the right prox sum to the right one. In turn, the robot turns in the opposite direction of the detected obstacle.

The switching between the different states is done with the measured values of the proximity sensors. If at least one of the four exterior sensor values is above a certain threshold, the robot switches to state 1. If all the proximity values are below another set threshold, the state switches back to 0. 

We experienced problems when merging the global and local navigation. Depending on the geometry and the placement of the 3d obstacle in relation to the global path, the robot would sometimes get stuck. As a matter of fact, it was trying to avoid the obstacle but, as soon as the prox value was below the threshold, it would turn back towards the obstacle when trying to follow the global path. This meant the thymio was constantly changing state and switching from two opposite objectives: turning one direction to avoid the obstacle and turning the opposite direction to reach its goal. 

In order to tackle this error, when the robot switches back to state 0 from state 1, meaning when the obstacle is no longer detected, the global path towards the goal is recomputed with a starting point in front of the thymio. As a result, when the robot turned enough to avoid the obstacle, it goes straight to a new starting point and follows its new trajectory towards the goal, this time from a place far enough from the obstacle. In turn, the path given as input to the global_nav() function when the robot is in state 0 is either the original shortest path calculated during the initialization outside the main loop, or the new path calculated when moving away from a 3d obstacle. 

<font color='red'>INCLUDE FIGURE LOCAL NAVIGATION</font>

## 8. Code

## 9. References

- https://docs.opencv.org/4.x/d5/dae/tutorial_aruco_detection.html
- https://github.com/TaipanRex/pyvisgraph
- Basics of Mobile Robotics
- http://cs.indstate.edu/~rjaliparthive/dijkstras.pdf
- https://courses.lumenlearning.com/waymakermath4libarts/chapter/shortest-path/